In [1]:
# 必要ライブラリのインストール
from PIL import Image
import numpy as np

In [2]:
def MSE(x, y):
    l = len(x)
    sum = 0.0
    for i in range(l):
        diff = float(x[i]) - float(y[i])
        sum += (diff * diff)
    return sum / l

In [12]:
img1 = Image.open('c1b_approx4/c1b_16_16_0.5_128_4.tga')
img2 = Image.open('ref/c1b_ref.tga')
img1_gray = img1.convert('L')
img2_gray = img2.convert('L')
img1_gray_arr = np.array(img1_gray)
img2_gray_arr = np.array(img2_gray)

In [5]:
#coding: utf-8
import numpy as np
import matplotlib.pyplot as plt

"""
参考:
[1]『画像処理とパターン認識入門』酒井幸市 著
[2] scipy.fftpack.dct http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.fftpack.dct.html
"""

class DCT:
	def __init__(self,N):
		self.N = N	# データ数．
		# 1次元，2次元離散コサイン変換の基底ベクトルをあらかじめ作っておく
		self.phi_1d = np.array([ self.phi(i) for i in range(self.N) ])
		
		# Nが大きいとメモリリークを起こすので注意
		# MNISTの28x28程度なら問題ない
		self.phi_2d = np.zeros((N,N,N,N))
		for i in range(N):
			for j in range(N):
				phi_i,phi_j = np.meshgrid(self.phi_1d[i],self.phi_1d[j])
				self.phi_2d[i,j] = phi_i*phi_j

	def dct(self,data):
		""" 1次元離散コサイン変換を行う """
		return self.phi_1d.dot(data)
	
	def idct(self,c):
		""" 1次元離散コサイン逆変換を行う """
		return np.sum( self.phi_1d.T * c ,axis=1)
	
	def dct2(self,data):
		""" 2次元離散コサイン変換を行う """
		return np.sum(self.phi_2d.reshape(N*N,N*N)*data.reshape(N*N),axis=1).reshape(N,N)
	
	def idct2(self,c):
		""" 2次元離散コサイン逆変換を行う """
		return np.sum((c.reshape(N,N,1)*self.phi_2d.reshape(N,N,N*N)).reshape(N*N,N*N),axis=0).reshape(N,N)
	
	def phi(self,k):
		""" 離散コサイン変換(DCT)の基底関数 """
		# DCT-II
		if k == 0:
			return np.ones(self.N)/np.sqrt(self.N)
		else:
			return np.sqrt(2.0/self.N)*np.cos((k*np.pi/(2*self.N))*(np.arange(self.N)*2+1))
		# DCT-IV(試しに実装してみた)
		#return np.sqrt(2.0/N)*np.cos((np.pi*(k+0.5)/self.N)*(np.arange(self.N)+0.5))

In [6]:
N = 8

def DCT_block(img_block):
    #N = 8			# データの次元は10x10とする
    dct = DCT(N)	# 離散コサイン変換を行うクラスを作成
    c = dct.dct2(img_block)	# 2次元離散コサイン変換
    return c

In [13]:
i0 = 0
j0 = 0
block1 = img1_gray_arr[i0 : i0 + N, j0 : j0 + N]
block2 = img2_gray_arr[i0 : i0 + N, j0 : j0 + N]
dct_block1 = DCT_block(block1)
dct_block2 = DCT_block(block2)

In [16]:
block1

array([[181, 182, 185, 183, 177, 180, 180, 186],
       [184, 176, 177, 174, 180, 182, 182, 186],
       [186, 180, 176, 193, 178, 177, 180, 189],
       [185, 181, 183, 178, 179, 181, 178, 178],
       [177, 182, 186, 175, 178, 177, 187, 179],
       [177, 184, 179, 184, 178, 185, 176, 179],
       [176, 184, 179, 179, 184, 185, 178, 193],
       [187, 185, 180, 191, 182, 190, 179, 187]], dtype=uint8)

In [17]:
block2

array([[182, 173, 178, 196, 167, 184, 180, 170],
       [178, 172, 175, 172, 169, 177, 180, 169],
       [173, 181, 188, 177, 176, 185, 181, 193],
       [177, 182, 181, 172, 179, 186, 185, 186],
       [173, 174, 179, 173, 189, 176, 173, 175],
       [177, 184, 172, 190, 181, 174, 181, 171],
       [181, 185, 203, 173, 181, 174, 178, 176],
       [181, 190, 185, 176, 178, 194, 177, 174]], dtype=uint8)

In [14]:
for i in range(N):
    for j in range(N):
        print(dct_block1[i, j], end = ' ')
    print('')

2896.6875 -37.28957072103458 0.47517215329900253 -11.44971723707215 -3.636488315326268 -2.221594426390027 -0.6878292777469674 0.6652248494967807 5.812499999999931 6.524379007793215 13.05334483203218 12.638781315209343 -6.002813109910123 8.255807262538582 6.863994172889319 -1.0690961645848347 
2.8319593679116792 5.811003987327439 -3.390526536375199 15.468178065649766 0.44719649872199163 9.480546955830278 -6.514812322544396 -11.492917725934625 4.780071252888765 -4.054519939568944 0.018323201604413697 2.547680900269519 1.8665076866379335 7.22202010467829 0.3749948211968488 -4.85030909590769 
2.0132543747553697 -8.638331533527184 -11.074929608537417 -10.734848742932606 5.7754139215298395 2.773368667164849 11.4088913472323 -2.5121606647994192 -5.436212754591236 7.929295380153107 7.467822424445753 2.434980336766734 -6.703482856105618 -5.049617484661109 -3.399437648823995 1.435857386093835 
-9.692536280786499 8.517110284105797 -1.4166952938240325 7.287240529761931 7.210767552686148 5.05758236

In [15]:
for i in range(N):
    for j in range(N):
        print(dct_block2[i, j], end = ' ')
    print('')

2896.5 -40.25834508474827 2.9201225979490655 -13.228867542413298 -3.317642400508694 0.37905924573328775 -0.32494156960763476 0.10365918409311803 4.999999999999925 9.801541648412496 12.06825180287581 6.692436752155686 0.15652125206472167 -2.40587907132155 0.36172560273667376 1.5078464681139678 
2.718789531861262 4.7717061801934335 -6.541545027381402 10.618474769349874 -0.03432608496839862 10.913895710038034 -5.377437298297787 -12.289665696888356 4.582555081091119 -7.995031433493455 -3.6318350180435948 -1.9683770193539516 0.3324151814633586 0.9579859525081034 0.20642408276599916 -0.4348794486903991 
2.8035128758536234 -7.239326805436305 -12.004914547939975 -9.05463055670872 6.894359216893979 3.2584230018150118 9.189329591621263 -3.461684447129351 0.3691370114717203 7.021202342221233 8.90786972188563 5.342754441834887 -0.16900068024634685 -1.5580052508479043 0.038190302176943525 0.5235915625596568 
-9.257325767711222 8.075258758741654 1.798932758485627 7.190344732338076 8.216047043621854 